In [32]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
#0:black 1:white 2:blue 3:red 4:orange 5:green 6:yellow
BLACK = (0,0,0)
WHITE = (255,255,255)
BLUE = (255,0,0)
RED = (0,0,255)
GREEN = (0,255,0)
YELLOW = (0,255,255)
ORANGE = (0,128,255)

In [4]:
def imgshow(image):
    cv2.imshow("image",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def pltshow(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    plt.imshow(image)        

In [5]:
def cropsquare(pts,image):
    temp = np.zeros((len(pts),2))
    for i in range(len(pts)):
        temp[i] = pts[i][0] 
    pts = temp
        
    rect = np.zeros((4, 2), dtype = "float32")

    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
        
    tl,tr,br,bl = rect

    width1 = np.sqrt(((br[0]-bl[0])**2) + ((br[1]-bl[1])**2))
    width2 = np.sqrt(((tr[0]-tl[0])**2) + ((tr[1]-tl[1])**2))
    height1 = np.sqrt(((br[0]-tr[0])**2) + ((br[1]-tr[1])**2))
    height2 = np.sqrt(((bl[0]-tl[0])**2) + ((bl[1]-tl[1])**2))

    max_w = max(int(width1),int(width2))
    max_h = max(int(height1),int(height2))

    dst = np.array([[0, 0],[max_w - 1, 0],[max_w - 1, max_h - 1],[0, max_h - 1]], dtype = "float32")        
    M = cv2.getPerspectiveTransform(rect, dst)
        
    cropped = cv2.warpPerspective(image, M, (max_w, max_h))
    return cv2.fastNlMeansDenoising(cropped,None,10,7,21)

In [6]:
class cubepre():
    def __init__(self,imagelist):
        self.grid = [None,None,None,None,None,None]
        self.sq = [None,None,None,None,None,None]
        self.face = [None,None,None,None,None,None]
        self.edge = [None,None,None,None,None,None]
        self.img = imagelist
        height = 400
        scale = imagelist[0].shape[0]*(1/height)
        width = int(imagelist[0].shape[1]/scale)
        for k in range(len(imagelist)):
            self.img[k] = self.img[k].copy()
            self.sq_extract(self.img[k],k)
        self.get_faceval()
                       
    def sq_extract(self,image,i):
        temp = image.copy()
        temp = cv2.cvtColor(temp,cv2.COLOR_BGR2HSV)
        temp = cv2.GaussianBlur(temp,(11,11),0)        
        temp = cv2.Canny(temp,25,50)
        temp = cv2.GaussianBlur(temp,(5,5),0)
        temp = cv2.dilate(temp,np.ones([5,5]))
        contours,hierarchy = cv2.findContours(temp,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        areas = [cv2.contourArea(c) for c in contours]
        maxind = np.argmax(areas)
        cntmax = contours[maxind]
        sq_hull = cv2.convexHull(cntmax,True)
        epsilon = 0.1*cv2.arcLength(sq_hull,True)
        approx = cv2.approxPolyDP(sq_hull,epsilon,True)
        self.edge = cv2.drawContours(image.copy(),[approx],0,YELLOW)
        self.sq[i] = cropsquare(approx,self.img[i])

    def get_faceval(self):
        sqlist = self.sq.copy()
        pixel_b = np.zeros([3,3])
        pixel_g = np.zeros([3,3])
        pixel_r = np.zeros([3,3])
        size = 150
        start = int(size/6)
        step = int(size/3)
        for k in range(len(sqlist)):
            temp = cv2.resize(sqlist[k],(150,150))
            temp_blur = cv2.blur(temp,(3,3))
        
            for i in range(3):
                for j in range(3):
                    pixel_b[i,j],pixel_g[i,j],pixel_r[i,j] = temp_blur[start+(i*step),start+(j*step)]
                        
            matrix = self.colormap([pixel_b,pixel_g,pixel_r])
            self.update(matrix,k)
        
    def update(self,mat,k):
        self.grid[k] = mat
        self.face[k] = np.zeros((3,3,3),np.uint8)
        for i in range(3):
            for j in range(3):
                if mat[i,j] == 1:
                    self.face[k][i,j] = WHITE
                elif mat[i,j] == 2:
                    self.face[k][i,j] = BLUE
                elif mat[i,j] == 3:
                    self.face[k][i,j] = RED
                elif mat[i,j] == 4:
                    self.face[k][i,j] = ORANGE
                elif mat[i,j] == 5:
                    self.face[k][i,j] = GREEN
                elif mat[i,j] == 6:
                    self.face[k][i,j] = YELLOW
                else:
                    pass      
            
    def colormap(self,colors):
        mat = np.zeros([3,3])
        B,G,R = colors
        col = [BLACK,WHITE,BLUE,RED,ORANGE,GREEN,YELLOW]
        vals = [0,1,2,3,4,5,6]
        dist = np.zeros([len(col),1])
        for i in range(3):
            for j in range(3):                                
                for m in range(len(col)):
                    c = col[m]
                    dist[m] = abs(B[i][j]-c[0]) + abs(G[i][j]-c[1]) + abs(R[i][j]-c[2])   
                ind = np.argmin(dist)
                mat[i][j] = vals[ind]
        return mat

In [8]:
face0 = cv2.imread("/images/image1.jpeg",1)
face1 = cv2.imread("/images/image2.jpeg",1)
face2 = cv2.imread("/images/image3.jpeg",1)
face3 = cv2.imread("/images/image4.jpeg",1)
face4 = cv2.imread("/images/image5.jpeg",1)
face5 = cv2.imread("/images/image6.jpeg",1)
cube = cubepre([face0,face1,face2,face3,face4,face5])

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
pltshow(cube.face[5])

Common class for all faces of the cube.
uses canny for egde detection. 

In [56]:
face0 = cv2.imread(os.path.join(os.getcwd(),"/images/image1.jpeg"),1)

In [59]:
print(cwd)
face0

/media/srivenkat/New Volume F/Coding/Projects/Rubiks_cube_solver
